In [1]:
import numpy as np
# Fake data feature:
# quarter of the year [range(4)] (time of year dependency)
# market capitalization $B (company size dependency)
# industry [range(11)] of the SP500 sectors (type of company dependency)
# PE ratio (expected growth dependency)
# Analyst rating [range(3)] sell=0, hold=1, buy=2
fake_features = [
    (np.random.randint(0, 4),
      np.random.randint(0, 100), 
      np.random.randint(0, 11), 
      np.random.randint(0, 100), 
      np.random.randint(0, 3)) for _ in range(10)
]


In [2]:
fake_features

[(3, 4, 10, 8, 2),
 (2, 13, 0, 56, 0),
 (2, 47, 6, 47, 0),
 (1, 9, 7, 60, 1),
 (0, 53, 2, 25, 1),
 (2, 70, 7, 80, 2),
 (3, 20, 10, 67, 0),
 (0, 3, 2, 98, 2),
 (0, 91, 8, 52, 2),
 (2, 20, 4, 62, 0)]

In [9]:
import torch
from torch_geometric.data import Data

# Nodes (companies) and their features
# x = torch.tensor([feature_vector_for_company_1, feature_vector_for_company_2, ...], dtype=torch.float)
x = torch.tensor(fake_features, dtype=torch.float)

# Edge connections 
# edge_index = torch.tensor([
#     [source_node_1, source_node_2, ...],
#     [target_node_1, target_node_2, ...],
# ], dtype=torch.long)
edge_index = torch.tensor([
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    [1, 0, 3, 2, 5, 4, 7, 6, 9, 8],
], dtype=torch.long)

# Labels for nodes (whether stock price will go up (1) or down (0))
#y = torch.tensor([label_for_company_1, label_for_company_2, ...], dtype=torch.float)
y = torch.tensor([
    np.random.randint(0, 2) for _ in range(10)
], dtype=torch.long)

data = Data(x=x, edge_index=edge_index, y=y)


In [11]:
# Inspect the data shape
print(f"Node data shape: {x.shape}")
print(f"Edge data shape: {edge_index.shape}")
print(f"Label data shape: {y.shape}")   
data.num_classes = 1
data.num_nodes = 10


Node data shape: torch.Size([10, 5])
Edge data shape: torch.Size([2, 10])
Label data shape: torch.Size([10])


In [12]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class StockGNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 3)  # 5 -> 3
        self.conv2 = GCNConv(3, data.num_classes) # 3 -> 1

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StockGNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()

IndexError: Target 1 is out of bounds.